### Manually creating stim log for high frequency stim pulses

In [1]:
import io
import os
import sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import copy
import time

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

In [4]:
%matplotlib widget

# Load data

In [5]:
rec_folder = r'Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1'
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)

Experiment type: electrical and sensory stimulation


In [6]:
sync_data = exp._load_sync_dataset()

In [7]:
exp.stimulus_log_file

'Q:\\mouse771427\\THstim_d3_2025-01-24_10-40-15\\experiment1\\recording1\\all_stim_log.csv'

## Load the sync data

In [8]:
esweep_rising = sync_data.get_edges(keys=('estim_sweep',), kind='rising', units='seconds')
esweep_falling = sync_data.get_edges(keys=('estim_sweep',), kind='falling', units='seconds')
esync_rising = sync_data.get_edges(keys=('estim_sync',), kind='rising', units='seconds')
esync_falling = sync_data.get_edges(keys=('estim_sync',), kind='falling', units='seconds')

In [9]:
total_trials = len(esync_rising)
print('Total e-stim trials: %d' % total_trials)
num_sweeps = len(esweep_rising)
print('Number of e-stim sweeps: %d' % num_sweeps)
trials_per_sweep = int(total_trials / num_sweeps)
print('Trials per sweep: %d' % trials_per_sweep)

Total e-stim trials: 12000
Number of e-stim sweeps: 10
Trials per sweep: 1200


In [43]:
print(len(esweep_rising))

10


# Manual stim log: simple version (one event for each frequency session)

In [44]:
estim_log = pd.DataFrame([
    ['freq50Hz', 50, 20],
    ['freq50Hz', 50, 20],
    ['freq50Hz', 100, 20],
    ['freq100Hz', 50, 20],
    ['freq100Hz', 100, 20],
    ['freq50Hz', 30, 20],
    ['freq50Hz', 50, 20],
    ['freq50Hz', 50, 20],
    ['freq50Hz', 50, 20],
    ['freq50Hz', 100, 20],
], columns=['stim_type', 'parameter', 'duration'])

In [45]:
estim_log.tail()

,stim_type,parameter,duration
5,freq50Hz,30,20
6,freq50Hz,50,20
7,freq50Hz,50,20
8,freq50Hz,50,20
9,freq50Hz,100,20


In [46]:
len(estim_log)

10

In [47]:
sweeps = np.zeros(len(estim_log), dtype=int)
# sweeps[-4:-2] += 1
sweeps[-5:] += 1
print(sweeps)

[0 0 0 0 0 1 1 1 1 1]


In [48]:
estim_log['onset'] = esweep_rising
estim_log['offset'] = esweep_falling
# estim_log['sweep'] = np.zeros(len(estim_log), dtype=int)
estim_log['sweep'] = sweeps

In [49]:
estim_log.tail()

,stim_type,parameter,duration,onset,offset,sweep
5,freq50Hz,30,20,3054.06783,3074.06793,1
6,freq50Hz,50,20,3155.39423,3175.39434,1
7,freq50Hz,50,20,3395.78621,3415.78632,1
8,freq50Hz,50,20,3551.69760,3571.69771,1
9,freq50Hz,100,20,3665.24847,3685.24858,1


## Save the log

In [16]:
simplename = exp.stimulus_log_file[:exp.stimulus_log_file.find('estim_log')] + 'simple_' + exp.stimulus_log_file[exp.stimulus_log_file.find('estim_log'):]
print(simplename)

Q:\mouse771427\THstim_d2_2025-01-23_10-25-10\experiment1\recording1\simple_estim_log.csv


In [57]:
estim_log.to_csv(simplename, index=False)

# Make the full log

In [18]:
simple_log = pd.read_csv(
    exp.stimulus_log_file[:exp.stimulus_log_file.find('estim_log')] + 'simple_' + exp.stimulus_log_file[exp.stimulus_log_file.find('estim_log'):])

In [12]:
simple_log.head()

,stim_type,parameter,duration,onset,offset,sweep
0,freq50Hz,50,20,2320.34197,2340.34208,0
1,freq50Hz,50,20,2486.19348,2506.19359,0
2,freq50Hz,100,20,2585.67013,2605.67024,0
3,freq100Hz,50,20,2724.17095,2744.17105,0
4,freq100Hz,100,20,2837.27795,2857.27805,0


In [13]:
print(exp.stim_instruction_files['electrical'])

['Q:\\mouse771427\\THstim_d3_2025-01-24_10-40-15\\experiment1\\recording1\\stim_protocol_100Hz_20s.dat', 'Q:\\mouse771427\\THstim_d3_2025-01-24_10-40-15\\experiment1\\recording1\\stim_protocol_50Hz_20s.dat']


In [14]:
for indi, logrow in simple_log.iterrows():
    print(indi, logrow.stim_type, logrow.duration, logrow.sweep)

0 freq50Hz 20 0
1 freq50Hz 20 0
2 freq50Hz 20 0
3 freq100Hz 20 0
4 freq100Hz 20 0
5 freq50Hz 20 1
6 freq50Hz 20 1
7 freq50Hz 20 1
8 freq50Hz 20 1
9 freq50Hz 20 1


In [15]:
pathbase = os.path.dirname(exp.stim_instruction_files['electrical'][0])
print(pathbase)

Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1


In [16]:
mcs_dfs = []
for indi, logrow in simple_log.iterrows():
    filename = 'stim_protocol_' + logrow.stim_type.replace('freq', '') + '_' + str(logrow.duration) + 's.dat'
    efile = os.path.join(pathbase, filename)
    print(efile)
    with open(efile) as file:
        estim_txt = file.read() # open estim file and read in text
    estim = estim_txt.split('channel:') # split and read txt file
    d = {}
    for item in estim:
        if 'value' in item:
            key = 'channel'+str(item[1])
            d[key]=item.split('\n')
    channel1 = d['channel1'][1:]
    temp = pd.read_csv(io.StringIO('\n'.join(channel1)), delim_whitespace=True)
    temp['event_type'] = [logrow.stim_type] * len(temp)
    temp['event_ID'] = [indi] * len(temp)
    temp['parameter'] = [logrow.parameter] * len(temp)
    temp['sweep'] = [logrow.sweep] * len(temp)
    # temp['stim_type'] = ['biphasic'] * len(temp)
    mcs_dfs.append(temp)
    
ch1_df = pd.concat(mcs_dfs, axis=0, sort=False)
print(len(ch1_df))

Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_50Hz_20s.dat
Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_50Hz_20s.dat
Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_50Hz_20s.dat
Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_100Hz_20s.dat
Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_100Hz_20s.dat
Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_50Hz_20s.dat
Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_50Hz_20s.dat
Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_50Hz_20s.dat
Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_50Hz_20s.dat
Q:\mouse771427\THstim_d3_2025-01-24_10-40-15\experiment1\recording1\stim_protocol_50Hz_20s.dat
36020


In [17]:
ch1_df.tail()

,value,time,event_type,event_ID,parameter,sweep
2997,0,19600,freq50Hz,9,100,1
2998,-5,200,freq50Hz,9,100,1
2999,5,200,freq50Hz,9,100,1
3000,0,19600,freq50Hz,9,100,1
3001,0,60000000,freq50Hz,9,100,1


In [18]:
list_loggie = []
for i in range(1, len(ch1_df)):
    if ch1_df.value.iloc[i-1] < 0 and ch1_df.value.iloc[i] > 0:
        list_loggie.append([
            ch1_df.event_type.iloc[i], ch1_df.event_ID.iloc[i], 'biphasic', ch1_df.parameter.iloc[i],
            (ch1_df.time.iloc[i-1] + ch1_df.time.iloc[i]) * 1e-6, ch1_df.sweep.iloc[i]
        ])
estim_log = pd.DataFrame(list_loggie, columns=['event_type', 'event_ID', 'stim_type', 'parameter', 'duration', 'sweep'])

In [19]:
estim_log.head()

,event_type,event_ID,stim_type,parameter,duration,sweep
0,freq50Hz,0,biphasic,50,0.0004,0
1,freq50Hz,0,biphasic,50,0.0004,0
2,freq50Hz,0,biphasic,50,0.0004,0
3,freq50Hz,0,biphasic,50,0.0004,0
4,freq50Hz,0,biphasic,50,0.0004,0


In [22]:
print(len(estim_log))
print(len(esync_rising))

12000
12000


In [20]:
estim_log['onset'] = esync_rising
estim_log['offset'] = esync_falling

In [21]:
estim_log.head()

,event_type,event_ID,stim_type,parameter,duration,sweep,onset,offset
0,freq50Hz,0,biphasic,50,0.0004,0,2320.34197,2320.34237
1,freq50Hz,0,biphasic,50,0.0004,0,2320.36197,2320.36237
2,freq50Hz,0,biphasic,50,0.0004,0,2320.38197,2320.38237
3,freq50Hz,0,biphasic,50,0.0004,0,2320.40197,2320.40237
4,freq50Hz,0,biphasic,50,0.0004,0,2320.42197,2320.42237


## Save the log

In [30]:
all_stim_log = pd.concat([estim_log], axis=0, sort=False).sort_values(by='onset', axis=0, ignore_index=True)
all_stim_log = all_stim_log[['event_type', 'event_ID', 'stim_type', 'parameter', 'onset', 'offset', 'duration', 'sweep']]

In [31]:
all_stim_log.head()

,event_type,event_ID,stim_type,parameter,onset,offset,duration,sweep
0,freq50Hz,0,biphasic,10,1377.0518,1377.0522,0.0004,0
1,freq50Hz,0,biphasic,10,1377.0718,1377.0722,0.0004,0
2,freq50Hz,0,biphasic,10,1377.0918,1377.0922,0.0004,0
3,freq50Hz,0,biphasic,10,1377.1118,1377.1122,0.0004,0
4,freq50Hz,0,biphasic,10,1377.1318,1377.1322,0.0004,0


In [32]:
all_stim_log.tail()

,event_type,event_ID,stim_type,parameter,onset,offset,duration,sweep
25995,freq50Hz,10,biphasic,100,3267.88457,3267.88497,0.0004,0
25996,freq50Hz,10,biphasic,100,3267.90457,3267.90497,0.0004,0
25997,freq50Hz,10,biphasic,100,3267.92457,3267.92497,0.0004,0
25998,freq50Hz,10,biphasic,100,3267.94457,3267.94497,0.0004,0
25999,freq50Hz,10,biphasic,100,3267.96457,3267.96497,0.0004,0


In [33]:
all_stim_log.to_csv(exp.stimulus_log_file, index=False)

# Sensory stim

#### Set parameters and load sync file

In [23]:
VISUAL_STIMULI = ['fullscreen', 'circle', 'natural_scene']

In [24]:
sweep_rising = sync_data.get_edges(keys=('behavior_sweep',), kind='rising', units='seconds')
sweep_falling = sync_data.get_edges(keys=('behavior_sweep',), kind='falling', units='seconds')

vsync_rising = sync_data.get_edges(keys=('behavior_vsync',), kind='rising', units='seconds')
vsync_falling = sync_data.get_edges(keys=('behavior_vsync',), kind='falling', units='seconds')

pd_rising = sync_data.get_edges(keys=('photodiode',), kind='rising', units='seconds')
pd_falling = sync_data.get_edges(keys=('photodiode',), kind='falling', units='seconds')

if len(sweep_rising) == 0:
    between_sweep_inds = np.argwhere(np.diff(vsync_rising) > 10.).flatten()
    sweep_rising = np.concatenate(([vsync_rising[0]-1.], vsync_rising[between_sweep_inds+1]-1.))
    sweep_falling = np.concatenate((vsync_falling[between_sweep_inds]+1., [vsync_falling[-1]+1.]))

#### Load experiment log file

In [25]:
explogfile = pd.read_csv(exp.stim_instruction_files['sensory'][0])
explogfile.head()

,stim_type,parameter,ISI,duration,imtrial.thisRepN,imtrial.thisTrialN,imtrial.thisN,imtrial.thisIndex,thisRow.t,notes,trialStim.started,trialStim.stopped,Unnamed: 12
0,circle,white,4.453313,0.25,0,0,0,0,NaN,NaN,5.036973,5.291090,NaN
1,circle,white,4.214403,0.25,0,1,1,1,NaN,NaN,9.712236,9.962459,NaN
2,circle,white,4.321758,0.25,0,2,2,2,NaN,NaN,14.166089,14.416220,NaN
3,circle,white,3.916555,0.25,0,3,3,3,NaN,NaN,18.736722,18.986879,NaN
4,circle,white,3.751302,0.25,0,4,4,4,NaN,NaN,22.890106,23.140480,NaN


In [26]:
if 'display_stim' in list(explogfile.columns):
    print('This is the old style log')
    explogfile = explogfile.drop(labels='time', axis=1)

    ## Remove gray screen from log ##
    list_stim_type = []
    list_stim_param = []
    for i in range(1, len(explogfile)):
        if explogfile.stim_type.iloc[i-1] == 'gray_screen' and explogfile.stim_type.iloc[i] in VISUAL_STIMULI:
            list_stim_type.append(explogfile.stim_type.iloc[i])
            list_stim_param.append(explogfile.stim_parameter.iloc[i]) # to make sure order matches stim table
    stim_log = pd.DataFrame({
        'stim_type': list_stim_type,
        'parameter': list_stim_param,
    })
    
elif 'ISI' in list(explogfile.columns):
    print('This is the new style log')
    
    stim_log = explogfile.drop(
        labels=[
            'imtrial.thisRepN','imtrial.thisTrialN','imtrial.thisN','imtrial.thisIndex','thisRow.t','notes',
            'trialStim.started','trialStim.stopped','Unnamed: 12'
        ], axis=1
    )
    
else:
    print("Uh oh, tbd_eeg does not understand this log file.")

This is the new style log


#### Add photodiode times as onsets/offsets (if available)

In [27]:
if pd_rising.shape[0] == stim_log.shape[0]:
    stim_log['onset'] = pd_rising
    if pd_falling.shape[0] == stim_log.shape[0]:
        stim_log['offset'] = pd_falling
    else:
        print('Number of photodiode offsets does not match number of stimuli; estimating offsets.')
        stim_log['offset'] = stim_log['onset'] + stim_log['duration']
elif vsync_rising.shape[0] == stim_log.shape[0]:
    print('Number of photodiode onsets does not match number of stimuli; using vsync times instead (these are typically 50 ms early).')
    stim_log['onset'] = vsync_rising
    stim_log['offset'] = vsync_falling
else:
    print('Number of photodiode and vsync pulses do not match number of stimuli; manual stim_log creation is needed.')

#### Add duration (for old log) and sweep number

In [29]:
stim_log['sweep'] = np.zeros(len(stim_log), dtype=int) - 1
for i in range(len(sweep_rising)):
    # print('Sweep %d: %f to %f' % (i, sweep_start, sweep_end))
    stim_log.loc[(stim_log['onset'] > sweep_rising[i]) & (stim_log['onset'] < sweep_falling[i]), 'sweep'] = i

#### Check it

In [31]:
stim_log.tail()

,stim_type,parameter,ISI,duration,onset,offset,sweep
235,circle,white,4.204180,0.25,2264.86732,2265.11800,1
236,circle,white,3.944863,0.25,2269.30436,2269.55505,1
237,circle,white,3.663446,0.25,2273.49118,2273.74189,1
238,circle,white,4.111699,0.25,2277.39446,2277.64517,1
239,circle,white,3.799415,0.25,2281.74809,2281.99879,1


In [37]:
stim_log['event_type'] = ['visual'] * len(stim_log)
stim_log['event_ID'] = np.arange(len(stim_log))

In [47]:
stim_log = stim_log.drop(labels=['ISI'], axis=1)

In [48]:
stim_log.head()

,stim_type,parameter,duration,onset,offset,sweep,event_type,event_ID
0,circle,white,0.25,709.75129,710.01841,0,visual,0
1,circle,white,0.25,714.42184,714.67233,0,visual,1
2,circle,white,0.25,718.87557,719.12604,0,visual,2
3,circle,white,0.25,723.44606,723.69652,0,visual,3
4,circle,white,0.25,727.59952,727.85001,0,visual,4


# Put them all together and save

In [49]:
all_logs_list = [estim_log, stim_log] # estim is called estim_log and visual is called stim_log
all_stim_log = pd.concat(all_logs_list, axis=0, sort=False).sort_values(by='onset', axis=0)

In [51]:
all_stim_log.to_csv(exp.stimulus_log_file, index=False)

In [50]:
all_stim_log.tail()

,event_type,event_ID,stim_type,parameter,duration,sweep,onset,offset
11995,freq50Hz,9,biphasic,100,0.0004,1,3685.14857,3685.14897
11996,freq50Hz,9,biphasic,100,0.0004,1,3685.16857,3685.16897
11997,freq50Hz,9,biphasic,100,0.0004,1,3685.18857,3685.18897
11998,freq50Hz,9,biphasic,100,0.0004,1,3685.20857,3685.20897
11999,freq50Hz,9,biphasic,100,0.0004,1,3685.22857,3685.22897
